In [2]:
from pathlib import Path

data_folder: Path = Path("../data/")

raw_data_folder: Path = data_folder.joinpath("00_raw/")
draft_data_processing_folder: Path = data_folder.joinpath("10_draft_processing/")
aggregare_draft_data_folder: Path = data_folder.joinpath("20_aggregate_draft/")
prod_data_folder: Path = data_folder.joinpath("30_prod/")
features_data_folder: Path = data_folder.joinpath("40_features/")

In [14]:
import pandas as pd

dataset = pd.read_csv(
    filepath_or_buffer=prod_data_folder.joinpath("dataset.csv"),
    sep=",",
    encoding="UTF-8",
)
dataset["Date"] = pd.to_datetime(dataset["Date"], format="%Y-%m-%d")

inflation_rates = pd.read_csv(
    filepath_or_buffer=prod_data_folder.joinpath("inflation_rates.csv"),
    sep=",",
    encoding="UTF-8",
)
inflation_rates["Date"] = pd.to_datetime(inflation_rates["Date"], format="%Y-%m-%d")

sentiment_analysis = pd.read_csv(
    filepath_or_buffer=prod_data_folder.joinpath("sentiment_analysis.csv"),
    sep=",",
    encoding="UTF-8",
)
sentiment_analysis["Date"] = pd.to_datetime(sentiment_analysis["Date"], format="%Y-%m-%d")

In [76]:
from datetime import date

starting_date: date = date(2015, 1, 1)
ending_date: date = date(2016, 12, 31)

filtered_dataset = dataset[(dataset["Date"] >= pd.to_datetime(starting_date)) & (dataset["Date"] <= pd.to_datetime(ending_date))].reset_index(drop=True)
filtered_inflation_rates = inflation_rates[(inflation_rates["Date"] >= pd.to_datetime(starting_date)) & (inflation_rates["Date"] <= pd.to_datetime(ending_date))].reset_index(drop=True)
filtered_sentiment_analysis = sentiment_analysis[(sentiment_analysis["Date"] >= pd.to_datetime(starting_date)) & (sentiment_analysis["Date"] <= pd.to_datetime(ending_date))].reset_index(drop=True)

common_dates: set = set(filtered_dataset["Date"])
common_dates.intersection_update(set(filtered_inflation_rates["Date"]))
common_dates.intersection_update(set(filtered_sentiment_analysis["Date"]))

mask_dataset = filtered_dataset["Date"].isin(common_dates)
mask_inflation_rates = filtered_inflation_rates["Date"].isin(common_dates)
mask_sentiment_analysis = filtered_sentiment_analysis["Date"].isin(common_dates)

df = pd.concat((filtered_dataset[mask_dataset].reset_index(drop=True), filtered_inflation_rates[mask_inflation_rates].reset_index(drop=True)), axis=1)
df = df.T.drop_duplicates().T